# Assignment 6

### Part A

In [61]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [66]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [67]:
def exp_return(data, P):
    # Write your code here
    # exp_return(data, 40) should output 0.07500000000000002
    p_in_a_year = data ["Price in 1 Year"]
    prob = data ["Probability"]
    p_return = (p_in_a_year/ P) - 1
    exp_return = (p_return * prob). sum()
    return exp_return
exp_return(data, 40)

0.07500000000000002

Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [68]:
def volatility(data, P):
    # Write your code here
    # volatility(data,40) should output 0.21301408404140795
    p_in_a_year = data ["Price in 1 Year"]
    prob = data ["Probability"]
    p_return = (p_in_a_year/ P) - 1
    mean = p_return.mean()
    var = (prob * (p_return ** 2)).sum() - (mean ** 2)
    sd = np.sqrt(var)
    return sd
volatility(data,40)

0.21301408404140795

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [69]:
def interval(data, P):
    # Write your code here
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    Z = 1.96
    expected_return = exp_return (data, P)
    sd = volatility (data, P)
    N = len (data)
    margin_error = Z * (sd/ np.sqrt (N))
    lower = expected_return - margin_error
    upper = expected_return + margin_error
    return [lower,upper]
interval(data, 40)

[-0.16604812797447732, 0.3160481279744774]

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [70]:
def exp_return_capm(beta, market_return, rf):
    # Write your code here
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    a = rf + beta * (market_return -rf)
    return a
exp_return_capm(-0.6,0.1, 0.02)

-0.028

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [71]:
def clean_data(path):
    # Write your code here
    # This should yield data with 403316 rows × 4 columns
    data = pd.read_csv(path)
    data = data[pd.to_numeric(data['RET'], errors='coerce').notna()]
    security = data.groupby('PERMNO').size().reset_index(name = 'count')
    data = data [data ['PERMNO'].isin(security[security['count'] >= 500]['PERMNO'])]
    data = data [['PERMNO','date','CUSIP','RET']]
    return data
clean_data("monthlycrsp_mod6.csv")

PERMNO        date     CUSIP        RET
9027      10065  1970-01-30  00621210  -0.144375
9028      10065  1970-02-27  00621210   0.105882
9029      10065  1970-03-31  00621210   0.017857
9030      10065  1970-04-30  00621210  -0.078947
9031      10065  1970-05-29  00621210  -0.020952
...         ...         ...       ...        ...
3393558   84129  2022-08-31  98970110   0.016315
3393559   84129  2022-09-30  98970110  -0.075777
3393560   84129  2022-10-31  98970110   0.021235
3393561   84129  2022-11-30  98970110   0.005583
3393562   84129  2022-12-30  98970110  -0.051332

[403316 rows x 4 columns]

Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [59]:
def avg_return(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    output = pd.DataFrame(data={
        "CUSIP": [],
        "Average Return": []
    })
    counter = 0
    total = 0
    
    if not data.empty:
        CUSIP = data["CUSIP"].iloc[0]
        for index, row in data.iterrows():
            if row["CUSIP"] != CUSIP:
                if counter != 0:
                    output.loc[len(output.index)] = [CUSIP, total / counter]
                counter = 0
                total = 0
                CUSIP = row["CUSIP"]
            ret_value = pd.to_numeric(row["RET"], errors='coerce')
            if not pd.isna(ret_value):
                total += ret_value
                counter += 1
                
        if counter != 0:
            output.loc[len(output.index)] = [CUSIP, total / counter]
    return output

          CUSIP  Average Return
0      68391610       -0.141066
1      36720410        0.011788
2      05978R10        0.004069
3      39031810        0.007200
4      95815510        0.015354
...         ...             ...
35587  47737R10       -0.084977
35588  92870X30        0.089897
35589  78513510        0.005821
35590  82936G20       -0.002320
35591  88160R10        0.044138

[35592 rows x 2 columns]


In [49]:
def security_volatilities(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    
    output = pd.DataFrame(data = {
        "CUSIP" : [],
        "Volatility" : []
    })
    data = data[pd.to_numeric(data['RET'], errors='coerce').notnull()]
    
    v = data.groupby('CUSIP')['RET'].std().reset_index()
    v.columns = ['CUSIP', 'Volatility']
    return v

          CUSIP  Volatility
0      00002130    0.374791
1      00003210    0.137143
2      00016510    0.124194
3      00020910    0.107218
4      00030710    0.226461
...         ...         ...
35587  Y8977Y10    0.164824
35588  Y9233510    0.349538
35589  Y9369110    0.145510
35590  Y9384M10    0.073790
35591  Y9530810    0.261505

[35592 rows x 2 columns]


Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [54]:
def correlation(ret_data, vol_data):
    # Write your code here
    # correlation(rets, vols) should yield 0.44837090727059614
    merged_data = ret_data.merge(vol_data, on = 'CUSIP', how = 'inner')
    corr = merged_data ['Average return'].corr(merged_data['Volatility'])
    return corr